In [1]:
import random
from datamatrix import get_init_matrix
import KNN
import numpy as np

In [2]:
init_matrix = get_init_matrix()
recommender = KNN.KNearestNeighbour(k = 50, sim_treshold = 20, measure = "cosine", show = True, mean= True)
data = recommender.init_data(init_matrix)

test


In [3]:
def evaluation(init_matrix):
    number_of_evaluations = 2
    user_IDs_evaluation = random.sample(range(6040), number_of_evaluations)
    threshold = 3
    
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    
    for i in range(number_of_evaluations):
        target_user = user_IDs_evaluation[i]
        
        movie_ID_array = np.arange(1, 3953, 1)
        
        target_user_movie_ratings = init_matrix[target_user]
        target_user_rated_movies = np.vstack((movie_ID_array, target_user_movie_ratings))
        
        
        
        target_user_movieID_ratings = []
        for j in range(len(movie_ID_array)):
            if target_user_rated_movies[1][j] != 0:
                target_user_movieID_ratings.append(target_user_rated_movies[0][j])
                target_user_movieID_ratings.append(target_user_rated_movies[1][j])
        
        number_of_evaluated_movies = int(len(target_user_movieID_ratings)*0.2)
        
        target_user_movieID_ratings = np.array(target_user_movieID_ratings)
        target_user_movieID_ratings = target_user_movieID_ratings.reshape(int(len(target_user_movieID_ratings)/2), 2)

        target_user_movieID = target_user_movieID_ratings[:,0]
        target_movies = random.sample(list(target_user_movieID), number_of_evaluated_movies)
        target_movies = np.array(target_movies).astype(int)
        print(target_movies)

        estimated_target_movie_ratings_normalized = recommender.get_ratings(targetid=target_user, targetmovies=target_movies)

        for k in range(len(target_movies)):
            if init_matrix[target_user][target_movies[k]] > threshold and estimated_target_movie_ratings[k][1] > thresholdold:
                true_positive += 1
            elif init_matrix[target_user][target_movies[k]] < threshold and estimated_target_movie_ratings[k][1] > thresholdold:
                false_positive += 1
            elif init_matrix[target_user][target_movies[k]] > thresholdold and estimated_target_movie_ratings[k][1] < thresholdold:
                true_negative += 1
            else:
                false_negative += 1

    recall = (true_positive / (true_positive + true_negative))
    precision = (true_positive / (true_positive + false_positive))
    accuracy = ((true_positive + false_negative) / (true_negative + true_positive + false_negative + false_positive))


In [4]:
evaluation(init_matrix)

[2448  316 2858 1372  589 2094 1527 1584  318 2393 1590  788  527 2699
 2046 1356 1603 2571 1831  608]
Shape of targetid_movies (22,) 

Shape of targetid_ratings (22,) 

Shape of only_targets (6040, 22) 

Shape of no_empty_ids (4587, 22) 

Shape of not_user (4586, 22) 

Shape of final_matrix is: (0,) 

Shape of sim_and_id: (0,) 

Example of sim_and_id is: [] 



IndexError: too many indices for array